# GRID SEARCH CV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv(r"D:\NLP\Ex_Files_NLP_Python_ML_EssT\Exercise Files\Ch01\01_03\Start\SMSSpamCollection.tsv", sep = '\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
    'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1)
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

C:\Users\Prateek\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,23.943806,2.493251,0.788793,0.140378,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979354,0.979354,0.973944,0.969452,0.973046,0.975030,0.003837,1
8,51.918215,0.624766,1.192168,0.684838,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.980251,0.976661,0.973944,0.968553,0.971249,0.974132,0.004082,2
11,36.322740,1.668461,0.307606,0.057573,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976661,0.976661,0.973046,0.966757,0.973046,0.973234,0.003620,3
10,26.242649,2.814255,0.757404,0.694731,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.976661,0.973944,0.965858,0.972147,0.973234,0.004160,4
3,3.493177,0.494730,0.280890,0.072665,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.981149,0.976661,0.975741,0.963163,0.969452,0.973233,0.006268,5


In [7]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
    'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv = 5, n_jobs = -1)
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[0:5]

C:\Users\Prateek\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,20.924953,1.276948,0.638621,0.396586,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.973968,0.973046,0.970350,0.972147,0.973234,0.002089,1
11,33.004558,0.663704,0.278422,0.040003,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975763,0.976661,0.974843,0.969452,0.969452,0.973234,0.003141,2
10,20.548037,0.638486,0.364697,0.051794,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.977558,0.974843,0.966757,0.968553,0.973054,0.004554,3
6,3.640056,0.371935,0.227600,0.063576,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.973968,0.975763,0.970350,0.973046,0.968553,0.972336,0.002575,4
8,39.416571,0.603886,0.694891,0.103882,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.975763,0.973944,0.965858,0.967655,0.972156,0.004589,5
